In [16]:
import os

import netCDF4
from haversine import haversine

import src.inner.tide_gauge_station


def find_closest_grid_point(all_latitudes, all_longitudes, current_station):
    """
    For a given station, find the closest grid point in the oras5 dataset using the haversine formula
    :param all_longitudes: 
    :param all_latitudes: 
    :param current_station: 
    :return: 
    """
    slice_lat = all_latitudes[current_station.latitude + 10: current_station.latitude - 20]
    slice_lon = all_longitudes[current_station.longitude + 10: current_station.longitude - 20]
    for lat in slice_lat:
        for lon in slice_lon:
            haversine(current_station.latitude, current_station.longitude, lat, lon)
    pass


# find closes grid point for each station
output_path = "../output/create_simulated_dataset/"
stations = src.tide_gauge_station.read_and_create_stations("../data/rlr_monthly/filelist.txt",
                                                           os.path.join(output_path, "metadata.txt"))
# read one oras5 file to get the grid points
oras5 = netCDF4.Dataset("../data/Oras5/1958-1979/sossheig_control_monthly_highres_2D_195801_CONS_v0.1.nc")
print(oras5)
# print(oras5.variables)
# print(oras5.variables.keys())
lats = oras5.variables["nav_lat"][:]
lons = oras5.variables["nav_lon"][:]
print(f"min lat: {lats.min()}, max lat: {lats.max()}")
print(f"min lon: {lons.min()}, max lon: {lons.max()}")
for station in stations:
    find_closest_grid_point(lats, lons, station)
    print(station)









In [2]:
from matplotlib import pyplot as plt

rms_30_eofs = {100: 0.1598, 150: 0.1585, 200: 0.2028, 250: 0.1187, 300: 0.1075, 350: 0.1105, 400: 0.1281,
               450: 0.1316, 500: 0.1636, 550: 0.1243, 600: 0.1357, 650: 0.1466, 700: 0.1501}
# plot 
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(rms_30_eofs.keys(), rms_30_eofs.values())
ax.set_xlabel("No of stations")
ax.set_ylabel("RMS (in meters)")
plt.savefig("../../output/PCA/voronoi/rms_30eofs.png", dpi=400)
plt.close()


In [5]:
import os
import matplotlib
import matplotlib.pyplot as plt
import src.inner.tide_gauge_station

matplotlib.use('Cairo')
if not os.path.exists("../../output/plot_timeseries"):
    os.makedirs("../../output/plot_timeseries")
metadatapath = "../../output/plot_timeseries/metadata.txt"
stations = src.inner.tide_gauge_station.read_and_create_stations("../../data/rlr_monthly/filelist.txt",
                                                                 metadatapath)  # read stations

# plot timeseries of tide gauge station 1 
station_1 = stations[1]
timeseries = station_1.timeseries
timeseries_cleaned = {}
for date in timeseries.keys():
    if timeseries[date] != -99999:
        timeseries_cleaned[date] = timeseries[date]

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(list(timeseries_cleaned.keys()), list(timeseries_cleaned.values()), color="teal")
ax.set_xlabel("Year")
ax.set_ylabel("Sea level (in mm)")
plt.savefig("../../output/plot_timeseries/station_1.svg", dpi=400)
plt.close()



In [8]:

import os
import src.inner.tide_gauge_station

# calculate number of valid data points across all stations timeseries
valid_data_points = 0
if not os.path.exists("../../output/plot_timeseries"):
    os.makedirs("../../output/plot_timeseries")
metadatapath = "../../output/plot_timeseries/metadata.txt"
stations = src.inner.tide_gauge_station.read_and_create_stations("../../data/rlr_monthly/filelist.txt",
                                                                 metadatapath)  # read stations
earliest_date = 1807.0417
latest_date = 2023.9583
# calculate number of missing values if all timeseries where to start at the earliest date and end at the latest date
no_missing_values = 0
for station in stations.values():
    timeseries = station.timeseries
    if len(timeseries) == 0:
        continue
    current_earliest_date = min(timeseries.keys())
    current_latest_date = max(timeseries.keys())
    for date in timeseries.keys():
        if timeseries[date] != -99999:
            valid_data_points += 1
        else:
            no_missing_values += 1
            if date < earliest_date:
                earliest_date = date
            if date > latest_date:
                latest_date = date
    no_missing_values += int((latest_date - current_latest_date) * 12) + int(
        (current_earliest_date - earliest_date) * 12)
print(f"Total valid data points: {valid_data_points}")
print(f"Average of valid data points per station: {valid_data_points / len(stations)}")
print(f"Earliest date: {earliest_date}")
print(f"Latest date: {latest_date}")
print(f"Number of missing values: {no_missing_values}")


Total valid data points: 706000
Average of valid data points per station: 445.42586750788644
Earliest date: 1807.0417
Latest date: 2023.9583
Number of missing values: 3391781


In [6]:
# plot rms values for reconstruction 
import matplotlib.pyplot as plt
import matplotlib

matplotlib.use('Cairo')
# set font size to 20
plt.rcParams.update({'font.size': 20})

# voronoi rms values
# rms_values = {100: 0.010123966802054275, 150: 0.009807647155841584, 200: 0.009155430208419977,
#               250: 0.008772708054281734, 300: 0.008773301823678473, 350: 0.008662121568280626,
#               400: 0.008990400503010964, 450: 0.009647321464568778, 500: 0.009558559797518962,
#               550: 0.009934817503266934, 600: 0.00998536744485348, 650: 0.010239007752516264,
#               700: 0.010477984463352425}
# 
# # section clustering 
# rms_values = {"100": 0.007736578500638809, "150": 0.008449788967305432, "200": 0.010995570288404487,
#               "250": 0.010100540919944138, "300": 0.009591489235576869, "350": 0.009774225561659824,
#               "400": 0.009798436236852436, "450": 0.009405158181798273, "500": 0.009165021788743557,
#               "550": 0.008808101275692802, "600": 0.009408070070172766, "650": 0.009416871818956039,
#               "700": 0.009844928828835775}

# normal clustering
rms_values = {"5": 0.01185586943356649, "10": 0.011460480587120958, "15": 0.010608438538163735,
              "20": 0.010580701672853823, "25": 0.010459958721936625, "30": 0.00958083810908065,
              "35": 0.00994762662884828, "40": 0.009424243271147041, "45": 0.010131756184639617,
              "50": 0.009739135848099602, "55": 0.009853429674342325, "60": 0.009400187264772576,
              "65": 0.00998535465856468, "70": 0.010127576982962745, "75": 0.009632241834844358,
              "80": 0.00980483357883341, "85": 0.010163861975817935, "90": 0.011571113213915027,
              "95": 0.010816039385113082, "100": 0.0108011764585149}

fig, ax = plt.subplots(figsize=(20, 12))
ax.plot(rms_values.keys(), rms_values.values(), color="teal", linewidth=4)
# inverse x axis for radius (to be comparable with the other plots, bc higher radius = fewer stations)
ax.invert_xaxis()
ax.set_xlabel("Radius")
ax.set_ylabel("RMSE [m]")
plt.savefig(f"../../output/PCA/normal_clustering_rms.svg")
plt.close()

In [1]:
# number of flagged values 
import src.inner.tide_gauge_station

path = "../../data/rlr_monthly/filelist.txt"
metadatapath = "../../output/metadata.txt"
stations = src.inner.tide_gauge_station.read_and_create_stations(path, metadatapath)

In [3]:
# read all rms_stations.txt files across the clustering sizes and take the mean, max and min and plot them 
import matplotlib.pyplot as plt
import matplotlib
import os

matplotlib.use('Cairo')
# set font size to 20
plt.rcParams.update({'font.size': 20})
path = ("../../output/PCA/voronoi/10_eofs/")
cluster_sizes = [150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700]
min_rms = []
overall_min_rms = 1
overall_min_station = 0
overall_min_cluster_size = 0
overall_max_rms = 0
overall_max_station = 0
overall_max_cluster_size = 0
max_rms = []
mean_rms = []
for size in cluster_sizes:
    current_path = os.path.join(path, f"1992_2023/{size}/", "rms_stations.txt")
    with open(current_path, "r") as file:
        # the rms value is the second value in each file
        rms_values = []
        for line in file:
            current_values = line.split()
            current_rms = float(current_values[1])
            rms_values.append(current_rms)
            if current_rms < overall_min_rms:
                overall_min_rms = current_rms
                overall_min_station = current_values[0]
                overall_min_cluster_size = size
            if current_rms > overall_max_rms:
                overall_max_rms = current_rms
                overall_max_station = current_values[0]
                overall_max_cluster_size = size
        min_rms.append(min(rms_values))
        max_rms.append(max(rms_values))
        mean_rms.append(sum(rms_values) / len(rms_values))

print(f"Overall min rms: {overall_min_rms}, station: {overall_min_station}, cluster size: {overall_min_cluster_size}")
print(f"Overall max rms: {overall_max_rms}, station: {overall_max_station}, cluster size: {overall_max_cluster_size}")
# save min, max and mean rms values
with open(os.path.join(path, "min_max_avg_rms.txt"), "w") as file:
    file.write("Min RMSE \n")
    for rms in min_rms:
        file.write(f"{rms}, ")
    file.write("\n")
    file.write("Max RMSE \n")
    for rms in max_rms:
        file.write(f"{rms}, ")
    file.write("\n")
    file.write("Avg RMSE \n")
    for rms in mean_rms:
        file.write(f"{rms}, ")

# plot 
fig, ax = plt.subplots(figsize=(20, 12))
ax.plot(cluster_sizes, min_rms, label="Min RMSE", color="teal", linewidth=4)
ax.plot(cluster_sizes, max_rms, label="Max RMSE", color="goldenrod", linewidth=4)
ax.plot(cluster_sizes, mean_rms, label="Avg RMSE", color="firebrick", linewidth=4)
ax.set_xlabel("Number of clusters")
ax.set_ylabel("RMSE [m]")
ax.legend()
plt.savefig(os.path.join(path, "min_max_avg_rms.svg"))
plt.close()

Overall min rms: 0.049477899589553336, station: 960, cluster size: 700
Overall max rms: 0.42973197316401895, station: 57, cluster size: 350


In [1]:
# read in stations and calculate for every ten year period how many stations have data
import src.inner.tide_gauge_station

path = "../../data/rlr_monthly/filelist.txt"
metadatapath = "../../output/metadata.txt"
stations = src.inner.tide_gauge_station.read_and_create_stations(path, metadatapath)
start_year = 1807
end_year = 2024
step_length = 10
time_steps = [(i, i + step_length) for i in range(start_year, end_year, step_length)]
no_of_stations_all_intervals = []
for time_step in time_steps:
    start, end = time_step
    no_stations = 0
    for station in stations.values():
        timeseries = station.timeseries
        for date in timeseries.keys():
            if start <= date <= end:
                no_stations += 1
                break
    no_of_stations_all_intervals.append(no_stations)
print(no_of_stations_all_intervals)
print(f"Avg no of stations per interval: {sum(no_of_stations_all_intervals) / len(no_of_stations_all_intervals)}")



[2, 2, 3, 4, 10, 24, 32, 46, 82, 104, 141, 175, 223, 280, 483, 715, 854, 927, 1010, 979, 1030, 917]
Avg no of stations per interval: 365.59090909090907


In [1]:
import json
import os
import src.inner.tide_gauge_station
import matplotlib.pyplot as plt
import matplotlib

matplotlib.use('Cairo')
# set font size to 20
plt.rcParams.update({'font.size': 20})

# plot 10 year intervals clustering solution + how many stations have data in that interval
clustering_path = "../../output/enforce_overlap/"
station_path = "../../data/rlr_monthly/filelist.txt"
metadatapath = "../../output/metadata.txt"
stations = src.inner.tide_gauge_station.read_and_create_stations(station_path, metadatapath)
start_year = 1807
end_year = 2024
step_length = 10
time_steps = [(i, i + step_length) for i in range(start_year, end_year, step_length)]
radii = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
no_of_stations_per_time_step = {}
no_of_clusters_per_interval_per_radius = {}
for radius in radii:
    no_of_clusters_per_interval_per_radius[radius] = {}
for time_step in time_steps:
    start, end = time_step
    no_of_stations = 0
    for station in stations.values():
        timeseries = station.timeseries
        for date in timeseries.keys():
            if start <= date <= end:
                no_of_stations += 1
                break
    no_of_stations_per_time_step[start] = no_of_stations
    no_of_stations_per_time_step[(end - 1)] = no_of_stations
    for radius in radii:
        no_stations_clustering = 0
        path = os.path.join(clustering_path, f"{start}_{end}/solution_{radius}.json")
        with open(path, "r") as file:
            current_clustering = json.load(file)
            no_stations_clustering = len(current_clustering)
        no_of_clusters_per_interval_per_radius[radius][start] = no_stations_clustering
        no_of_clusters_per_interval_per_radius[radius][(end - 1)] = no_stations_clustering

colors = ["teal", "firebrick", "goldenrod", "purple", "forestgreen", "mediumblue", "chocolate", "tomato",
          "yellowgreen", "dodgerblue", "slategrey", "darkorchid"]
# plot
fig, ax = plt.subplots(figsize=(20, 12))
color_index = 0
for radius in radii:
    ax.plot(no_of_stations_per_time_step.keys(), no_of_clusters_per_interval_per_radius[radius].values(),
            label=f"Radius: {radius}mm", linewidth=4, color=colors[color_index])
    color_index += 1
ax.plot(no_of_stations_per_time_step.keys(), no_of_stations_per_time_step.values(), label="No of stations", linewidth=4,
        color="red", linestyle="--")
ax.set_xlabel("Time interval")
ax.set_ylabel("Number of centers")
ax.legend()
plt.savefig(os.path.join(clustering_path, "no_of_clusters_per_interval.svg"))
plt.close()

